In [5]:
import numpy as np
import matplotlib.pyplot as plt

from astropy.table import Table, join, Column
import astropy.units as u 
from astropy.coordinates import SkyCoord
from astropy.io import ascii

import astroquery
from astroquery.mast import Catalogs
from astroquery.mast import Observations

import glob, re

% matplotlib inline

import matplotlib as mpl
from matplotlib import rc

mpl.style.use('seaborn-colorblind')

#To make sure we have always the same matplotlib settings
#(the ones in comments are the ipython notebook settings)

mpl.rcParams['figure.figsize']=(12.0,9.0)    #(6.0,4.0)
mpl.rcParams['font.size']=18               #10 
mpl.rcParams['savefig.dpi']= 200             #72 
mpl.rcParams['axes.labelsize'] = 16
mpl.rcParams['axes.labelsize'] = 16
mpl.rcParams['xtick.labelsize'] = 12
mpl.rcParams['ytick.labelsize'] = 12

colours = mpl.rcParams['axes.prop_cycle'].by_key()['color']


In [14]:
fname = '../data/bh_table.csv'
bhs = Table.read(fname,delimiter='\t')
bhs.rename_column('Name (Counterpart)','Name')
bhs.rename_column('RA (h:m:s)','RA')
for key in bhs.keys():
    if 'DEC' in key: 
        bhs.rename_column(key,'Dec')

In [15]:
bhs

ID,Name,RA,Dec,l (deg),b (deg),Magnitude Outburst (AB),Mag. Quies. (AB),d (kpc),fxpeak [2-10] (erg/s/cm2),Porb (h)
int64,str42,str11,str12,float64,float64,str15,str13,str9,float64,str15
60,MAXI J1535-571GRB 170902A,15:35:19.71,-57:13:47.58,323.7240835,-1.12886,i’=21.8±0.2,--,--,1.44e-07,--
59,IGR J17454-2919,17:45:27.69,-29:19:53.83,359.6444268,-0.1765105,--,J=17.137,--,2.78e-10,--
58,IGR J17451-3022,17:45:06.72,-30:22:43.30,358.7115137,-0.6579992,--,--,--,1.9e-10,∼6.284±0.001
57,MAXI J1828-249,18:28:58.07,-25:01:45.88,8.1151012,-6.5448459,r'=16.9±0.1,--,--,4.58e-09,--
56,SWIFT J1753.7-2544,17:53:39.85,-25:45:14.20,3.6476003,0.1035382,K (not AB)∼16.5,--,--,7.09e-09,--
55,SWIFT J174510.8-262411,17:45:10.85,-26:24:12.60,2.1108255,1.4032659,i'∼17.6,r'>23.1±0.5,<7,2.9e-08,≤21
54,SWIFT J1910.2-0546 MAXI J1910-057,19:10:22.80,-05:47:55.92,29.9025737,-6.8440294,r'=15.7±0.1,--,--,1.95e-08,>6.2
53,MAXI J1305-704,13:06:55.30,-70:27:05.11,304.2379729,-7.6189888,uvw1=17.23±0.04,--,--,1.03e-09,9.74±0.04
52,MAXI J1836-194,18:35:43.45,-19:19:10.48,13.9456207,-5.3543217,V=16.33±0.08,r>23.474,7±3,1.01e-09,<4.9


In [38]:
for j in range(len(bhs)):
    ra, dec = bhs[j]['RA','Dec']
    obsTable = Observations.query_region("%s %s "  % (ra, dec),radius=2*u.arcmin)
    m = [obsTable['obs_collection']=='TESS']
    if np.sum(m) != 0:
        print(bhs['Name'][j])
#         print(obsTable[m])

/anaconda2/envs/py36/lib/python3.6/site-packages/astroquery/utils/commons.py:171: UserWarning: Coordinate string is being interpreted as an ICRS coordinate provided in degrees.
  warnings.warn("Coordinate string is being interpreted as an "


MAXI J1535-571GRB 170902A
MAXI J1305-704
MAXI J1543-564
XTE J1652-453
SWIFT J1539.2-6227
XTE J1726-476 IGR J17269-4737
IGR J17098-3628
IGR J17091-3624 SAX J1709.1-3624
XTE J1650-500
XTE J1550-564 V381 Nor
GRO J1655-40 N Sco 1994 = V1033 Sco
GRS 1009-45 N Vel 1993 = MM Vel
GRS 1124-684  N Mus 1991 = GU Mus
GS 1354-64 BW Cir
3A 1524-617 KY TrA
1H 1659-487 GX 339-4 = V821 Ara
4U 1543-475 IL Lup
4U 1630-472 Nor X-1
Cen X-2


### Our favourite is probably Cen X-2 - let's try and get TESS data for this

In [22]:
obsTable

dataproduct_type,obs_collection,instrument_name,project,filters,wavelength_region,target_name,target_classification,obs_id,s_ra,s_dec,proposal_id,proposal_pi,calib_level,t_min,t_max,t_exptime,em_min,em_max,obs_title,t_obs_release,proposal_type,sequence_number,s_region,jpegURL,dataURL,dataRights,mtFlag,srcDen,obsid,objID,distance
str5,str5,str10,str4,str4,str7,str12,str1,str19,float64,float64,str3,str14,int64,float64,float64,float64,float64,float64,str1,float64,str3,int64,str1855,str117,str112,str6,bool,float64,str11,str11,float64
image,TESS,Photometer,TESS,TESS,Optical,TESS FFI,--,tess-s0002-3-2,11.941583568308259,-60.01484789519171,N/A,"Ricker, George",3,58353.60693574,58381.00244957,1425.599483,600.0,1000.0,--,58458.58334,--,2,POLYGON 32.477102 -62.227635 16.359598 -54.028976 16.35952533577985 -54.029012055442443 16.359349 -54.028886 16.358902021471508 -54.029107783373995 16.358333 -54.028701 16.358143006249527 -54.028795267922291 16.357944 -54.028653 16.357701334165547 -54.028773400696423 16.35767 -54.028751 16.357649226764067 -54.028761306802693 16.357646 -54.028759 16.3576457610504 -54.028759118555833 16.35764 -54.028755 16.357637754696544 -54.028756114015245 16.357632 -54.028752 16.35762904490803 -54.028753466179175 16.35762 -54.028747 16.357618699899792 -54.028747645049521 16.357615 -54.028745 16.357613225525821 -54.02874588041103 16.357605 -54.02874 16.35760440968772 -54.028740292885225 16.357604 -54.02874 16.357599382815568 -54.028742290829605 16.357571 -54.028722 16.357567222708628 -54.0287238741137 16.357566 -54.028723 16.357562454574193 -54.028724759072112 16.357553 -54.028718 -1.9922784435604575 -60.402554942454479 -2.6506919999999354 -60.549816 -2.6506919872104859 -60.549816015453217 -2.6507010000000264 -60.549818 -2.6506979621577065 -60.549821670514596 -2.6507040000000188 -60.549823 -2.6507025870942451 -60.549824707163019 -2.6507129999999961 -60.549827 -2.6507028909149537 -60.549839214445221 -2.6507110000000011 -60.549841 -2.6507088998290254 -60.549843537559696 -2.6507110000000011 -60.549844 -2.6507042242541714 -60.549852186888906 -2.650716999999986 -60.549855 10.956694 -70.958009 10.956723981140163 -70.958002070219266 10.957094 -70.958177 10.957268418406466 -70.958136684732608 10.957451 -70.958223 10.958091585428384 -70.958074931029088 10.958115 -70.958086 10.958680721213121 -70.957955231779223 10.958786 -70.958005 10.959261193991086 -70.957895154487673 10.959868 -70.958182 10.960255241896879 -70.958092481933249 10.960415 -70.958168 32.476887 -62.227872 32.476795609917588 -62.227840092698457 32.477102 -62.227635 32.477102 -62.227635,--,--,PUBLIC,False,--,17000072397,17000600954,0.0
image,GALEX,GALEX,AIS,NUV,UV,AIS_392_1_25,--,6384708056189501440,14.9121797546718,-56.9874708963636,--,--,2,55892.521053240744,55892.59054398148,179.0,169300000000.0,300700000000.0,--,56189.28269,AIS,--,CIRCLE ICRS 14.91217975 -56.98747090 0.625,http://galex.stsci.edu/data/GR6/pipe/02-vsn/50392-AIS_392/d/01-main/0001-img/07-try/qa/AIS_392_sg25-xd-int_2color.jpg,http://galex.stsci.edu/data/GR6/pipe/02-vsn/50392-AIS_392/d/01-main/0001-img/07-try/AIS_392_sg25-nd-int.fits.gz,PUBLIC,--,5885.0,1000037698,1000037666,0.0
image,GALEX,GALEX,AIS,FUV,UV,AIS_392_1_36,--,6384708068000661504,16.2709085610128,-57.7125304705332,--,--,2,54330.37613425926,54332.36292824074,218.0,134000000000.0,180600000000.0,--,55435.11046,AIS,--,CIRCLE ICRS 16.27090856 -57.71253047 0.625,http://galex.stsci.edu/data/GR6/pipe/02-vsn/50392-AIS_392/d/01-main/0001-img/07-try/qa/AIS_392_sg36-xd-int_2color.jpg,http://galex.stsci.edu/data/GR6/pipe/02-vsn/50392-AIS_392/d/01-main/0001-img/07-try/AIS_392_sg36-fd-exp.fits.gz,PUBLIC,--,5885.0,1000037709,1000074148,0.0
image,GALEX,GALEX,AIS,NUV,UV,AIS_392_1_36,--,6384708068000661504,16.2709085610128,-57.7125304705332,--,--,2,54330.37613425926,54332.36292824074,218.0,169300000000.0,300700000000.0,--,55435.11046,AIS,--,CIRCLE ICRS 16.27090856 -57.71253047 0.625,http://galex.stsci.edu/data/GR6/pipe/02-vsn/50392-AIS_392/d/01-main/0001-